# Step 1: Import the libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import cv2
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib parameters
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")

# Step 2: データ読み込み関数の定義
 - 動物と人間の顔画像を読み込むための2つの関数が定義されます。

In [9]:
def load_animal_data(data_dir="../data/af_data_new"):
    """Load animal image data (3 classes: cat, dog, wild)"""
    print("=== Data Collection and Preprocessing ===")
    print(f"Data source: {data_dir}")
    print("Preprocessing: RGB→Grayscale conversion, 128×128 resize, normalization (0-1)")
    
    data_path = Path(data_dir)
    animal_classes = ['cat', 'dog', 'wild']
    X, y = [], []
    
    for class_idx, class_name in enumerate(animal_classes):
        class_dir = data_path / class_name
        if not class_dir.exists():
            continue
            
        image_files = list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png"))
        print(f"{class_name}: {len(image_files)} images")
        
        for img_path in image_files:
            try:
                img = cv2.imread(str(img_path))
                if img is None:
                    continue
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (128, 128))
                img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                img_normalized = img_gray.flatten() / 255.0
                X.append(img_normalized)
                y.append(class_idx)
            except:
                continue
    
    print(f"Total data count: {len(X)} images")
    return np.array(X), np.array(y), animal_classes

def load_human_data(data_dir="../data/af_data_new"):
    """Load human image data (animal-like features)"""
    print("\n=== Animal-like Human Face Images ===")
    print("Collection method: AI image generation services")
    print("Labeling method: Filename-based (e.g., cat_human_01.jpg)")
    
    data_path = Path(data_dir) / "human_like_animal"
    if not data_path.exists():
        return np.array([]), np.array([]), []
    
    X_human, y_human, human_files = [], [], []
    class_mapping = {'cat': 0, 'dog': 1, 'wild': 2}
    
    image_files = list(data_path.glob("*.jpg")) + list(data_path.glob("*.png"))
    print(f"Human image count: {len(image_files)}")
    
    for img_path in image_files:
        try:
            filename = img_path.name
            animal_name = filename.split('_')[0].lower()
            if animal_name not in class_mapping:
                continue
            
            img = cv2.imread(str(img_path))
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (128, 128))
            img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img_normalized = img_gray.flatten() / 255.0
            
            X_human.append(img_normalized)
            y_human.append(class_mapping[animal_name])
            human_files.append(filename)
        except:
            continue
    
    for class_name in ['cat', 'dog', 'wild']:
        count = sum(1 for f in human_files if f.startswith(class_name))
        print(f"{class_name}-like: {count} images")
    
    return np.array(X_human), np.array(y_human), human_files

print("Data loading functions defined!")

Data loading functions defined!


# Step 3: データセットの読み込み
- 定義した関数を使って、実際に画像データを読み込む。各クラスの画像数や、データの形状（次元数）が出力される。


In [ ]:
# Load animal data
X, y, class_names = load_animal_data()
print(f"\nLoaded data shape: {X.shape}")
print(f"Classes: {class_names}")

# Load human data
X_human, y_human, human_files = load_human_data()

# Step 4: PCA（主成分分析）の実行
16,384次元の画像データを、110次元まで圧縮します。これにより、本質的な特徴を保ちながら計算コストを大幅に削減できます。

In [ ]:
# PCA implementation
print("=== PCA Implementation ===")
print("Library used: scikit-learn PCA")
print("Hyperparameter: n_components (number of principal components)")

n_components = 110
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

explained_variance = pca.explained_variance_ratio_.sum()
print(f"Number of components: {n_components}")
print(f"Explained variance ratio: {explained_variance:.4f}")
print(f"Eigenvalues (top 5): {pca.explained_variance_[:5]}")
print(f"PCA transformed data shape: {X_pca.shape}")

# Step 5: 固有顔の可視化
- 主成分は、いわば「平均的な顔」からの差分を表すパターンです。これを画像として表示することで、モデルが顔のどの部分（目、鼻、輪郭など）に注目しているのかを視覚的に理解できます。

In [ ]:
# Eigenfaces visualization
plt.figure(figsize=(12, 4))
for i in range(min(6, n_components)):
    plt.subplot(2, 3, i + 1)
    eigenface = pca.components_[i].reshape(128, 128)
    plt.imshow(eigenface, cmap='gray')
    plt.title(f'Eigenface {i+1}')
    plt.axis('off')
plt.suptitle('PCA Eigenfaces (Top 6 Components)')
plt.tight_layout()
plt.show()

# Step 6: PCAによる2次元可視化
- 次に、110次元に圧縮したデータをさらに2次元（第一主成分と第二主成分）まで落とし込み、各クラス（cat, dog, wild）がどのように分布しているかを散布図で確認します。これにより、クラスがどの程度分離可能か、視覚的な手がかりを得られます。

In [ ]:
# 2D scatter plot
plt.figure(figsize=(10, 8))
colors = ['red', 'green', 'blue']
for i, class_name in enumerate(class_names):
    mask = y == i
    plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
               c=colors[i], label=class_name, alpha=0.6, s=50)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('PCA 2D Visualization (Animal Images)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Step 7: 訓練データとテストデータの分割
- 次に、機械学習モデルの性能を正しく評価するため、データを「訓練用（80%）」と「テスト用（20%）」に分割します。モデルは訓練用データのみを見て学習し、テスト用データは学習後の性能評価にのみ使用します。

In [ ]:
# Data split
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training data: {X_train.shape[0]} samples")
print(f"Test data: {X_test.shape[0]} samples")
print(f"Training/Test split: {X_train.shape[0]/(X_train.shape[0]+X_test.shape[0]):.1%}/{X_test.shape[0]/(X_train.shape[0]+X_test.shape[0]):.1%}")

# Step 8: SVMの訓練とグリッドサーチ
- ここでは、以下の処理を行います。
- 特徴量の標準化: 各特徴量（主成分）のスケールを揃え、モデルの学習を安定させます。
- グリッドサーチ: SVMの性能を最大化するため、最適なハイパーパラメータ（C, gamma, kernel）の組み合わせを自動的に探索します。
- モデルの訓練: 最適なパラメータを使って、SVMモデルを訓練データに適合させます。
- この処理は計算に少し時間がかかる場合があります（数分程度）。

In [ ]:
# SVM implementation
print("=== SVM Implementation ===")
print("Library used: scikit-learn SVC")
print("Hyperparameters: C (regularization), gamma (RBF kernel), kernel (kernel function)")

# Feature standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Grid search
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

print("Running grid search...")
svm = SVC(probability=True) # probability=True is needed for confidence scores later
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Cross-validation score: {grid_search.best_score_:.4f}")

# Final model
svm_model = grid_search.best_estimator_
y_pred = svm_model.predict(X_test_scaled)

print("\nSVM model training complete!")

# Step 9: 動物画像の分類性能評価
- それでは、訓練したモデルが未知のデータ（テストデータ）に対してどれくらいの性能を発揮するかを評価します。ここでは、以下の2つの指標を使います。
- 分類レポート: クラスごとの適合率（Precision）、再現率（Recall）、F1スコアといった詳細な性能指標と、全体の正解率（Accuracy）を表示します。
- 混同行列: モデルがどのクラスをどのクラスと間違えやすいかを視覚的に示した行列です。対角線上の数字が大きいほど、正しく分類できていることを意味します。

In [ ]:
# Experiment 1: Animal classification accuracy
print("=== Experiment 1: Animal Classification Accuracy ===")
accuracy = accuracy_score(y_test, y_pred)
print(f"Overall accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Animal Classification')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Step 10: SVMの決定境界の可視化
- 次に、SVMがどのようにして3つのクラスを分類しているのか、その「境界線（決定境界）」を可視化します。これにより、モデルの分類ロジックを直感的に理解することができます。
- 背景色: モデルが予測するクラス領域を表します。（例: 赤い領域にプロットされた点は「cat」と予測される）
点: 実際のデータ点です。
- この可視化のために、再度2次元のPCAを行いますが、これはあくまで可視化用であり、実際のモデル（110次元）の性能とは異なります。

In [ ]:
# SVM decision boundary visualization
print("=== SVM Decision Boundary Visualization ===")

# 2D PCA for visualization
pca_2d = PCA(n_components=2)
X_2d = pca_2d.fit_transform(X)

# Standardization
scaler_2d = StandardScaler()
X_2d_scaled = scaler_2d.fit_transform(X_2d)

# SVM training for 2D
svm_vis = SVC(kernel='rbf', C=10, gamma='scale')
svm_vis.fit(X_2d_scaled, y)

# Create mesh
margin = 0.5
x_min, x_max = X_2d_scaled[:, 0].min() - margin, X_2d_scaled[:, 0].max() + margin
y_min, y_max = X_2d_scaled[:, 1].min() - margin, X_2d_scaled[:, 1].max() + margin

h = 0.02
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                    np.arange(y_min, y_max, h))

Z = svm_vis.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Visualization
plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)

colors = ['red', 'green', 'blue']
for i in range(len(class_names)):
    mask = y == i
    plt.scatter(X_2d_scaled[mask, 0], X_2d_scaled[mask, 1],
               c=colors[i], label=class_names[i], edgecolor='k', alpha=0.8)

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xlabel('First Principal Component (Scaled)')
plt.ylabel('Second Principal Component (Scaled)')
plt.title('SVM Decision Boundary (2D PCA Space)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Step 11: 人間画像の分類と評価
- いよいよ最終段階です。動物の顔で訓練したモデルが、動物的な特徴を持つ人間の顔をどのように分類するかを検証します。
- 前処理の適用: 人間画像にも、動物画像と全く同じPCAと標準化を適用します。
- 予測と確信度の計算: svm_model を使って、各人間画像を予測し、その際の「確信度（probability）」も計算します。
- 結果の表示: ファイル名、予測結果、正解、そしてモデルがどれくらいその予測に自信があるか（確信度）を一覧で表示します。

In [ ]:
# Experiment 2: Human image classification
if len(X_human) > 0:
    print("=== Experiment 2: Human Image Classification Results ===")
    
    # Apply same preprocessing pipeline
    X_human_pca = pca.transform(X_human)
    X_human_scaled = scaler.transform(X_human_pca)
    
    # Prediction
    predictions = svm_model.predict(X_human_scaled)
    probabilities = svm_model.predict_proba(X_human_scaled)
    
    # Display results
    print("Classification results:")
    results = []
    for i, (filename, pred) in enumerate(zip(human_files, predictions)):
        ground_truth = class_names[y_human[i]]
        predicted = class_names[pred]
        confidence = probabilities[i][pred]
        correct = "✓" if pred == y_human[i] else "✗"
        print(f"{filename}: {predicted} (Confidence: {confidence:.2%}) | Ground truth: {ground_truth} {correct}")
        
        results.append({
            'filename': filename,
            'predicted': predicted,
            'ground_truth': ground_truth,
            'correct': pred == y_human[i]
        })
    
    # Accuracy evaluation
    human_accuracy = accuracy_score(y_human, predictions)
    print(f"\nHuman image classification accuracy: {human_accuracy:.4f} ({human_accuracy*100:.2f}%)")
    
    # Classification report
    print("\nClassification report:")
    print(classification_report(y_human, predictions, target_names=class_names))
else:
    print("No human images found for evaluation")
    results = []
    human_accuracy = 0

# Step 12: 人間画像の分類結果の可視化
- 最後に、人間画像の分類結果を2つのグラフで可視化して、傾向を詳しく分析します。
- 混同行列: 人間画像において、どのクラスがどのクラスに間違われやすいかを表示します。
- 分類結果の棒グラフ: 全体として、どの動物の特徴（cat, dog, wild）に分類される傾向が強いかを示します。

In [ ]:
if len(X_human) > 0:
    # Confusion matrix for human classification
    cm_human = confusion_matrix(y_human, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_human, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix - Human Image Classification')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # Classification results summary
    class_counts = {'cat': 0, 'dog': 0, 'wild': 0}
    for result in results:
        class_counts[result['predicted']] += 1
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(class_counts.keys(), class_counts.values(), color=['red', 'green', 'blue'])
    plt.title('Human Image Classification Results')
    plt.xlabel('Animal Class')
    plt.ylabel('Number of Images')
    for bar, count in zip(bars, class_counts.values()):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
                f'{count}', ha='center', va='bottom')
    plt.show()
else:
    print("No human images to visualize")
